In [6]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import bomeba0 as bmb

In [7]:
pdb = 'structures/alanine_beta.pdb'
#Protein coordinates
XYZ = np.genfromtxt(pdb)[:,5:8]
#Selection coordinates
l = np.genfromtxt(pdb)[[0,5],:][:,5:8]
m = np.genfromtxt(pdb)[[1,6,4,7,8,9,2,3],:][:,5:8]
n = np.genfromtxt(pdb)[[10,11,14,15,16,17,18,19],:][:,5:8]
o = np.genfromtxt(pdb)[[12,13,20,25],:][:,5:8]
p = np.genfromtxt(pdb)[[21,22,23,24,26,27,28,29],:][:,5:8]
selec = [l, m, n, o, p]

In [ ]:
pdb = bmb.Protein('GAD')
#len(pdb.sequence)
#pdb.coords[4 : pdb._offsets[1]]#cadena lateral residuo 1
#pdb.at_coords(1)

In [ ]:
def fragments(prot):
    """
    Takes a Protein object and returns a list.
    of it's fragmented coordinates.
    ----------------
    Parameters
    ----------------
    prot: Protein object
    
    -----------------
    Returns
    -----------------
    fragments: List of selected coordinates.
    """
    fragments = [np.vstack((prot.at_coords(1, 'N'), prot.at_coords(1, 'H')))]
    
    for i in range(1,len(prot.sequence)-1):
        fragments.append(np.vstack((pdb.at_coords(i, 'CA'), pdb.at_coords(i, 'H'))))#+cadena lateral
        
        
        #fragments.append(np.vstack((prot.at_coords(i, 'C'), prot.at_coords(i, 'O'))))#, 
        #np.vstack((prot.at_coords(i+1, 'N'), prot.at_coords(i+1, 'H')))))
    
    return fragments
fragments(pdb)

In [ ]:
class Node:    
    
    """Constructor to create a new node"""
    
    def __init__(self, obb, parents, children):
        self.obb = obb 
        self.parents = parents
        self.children = children
        
    def __str__(self):
        return '%s    %s    %s' % (self.obb, self.parents, self.children)
        
    def Vertex_OBB(XYZ):
        """
        Computes the 8 vertex of the Oriented Bounding Box
        -----------
        Parameters:
        -----------
        XYZ : array of shape (n,3). Contains the coordinates of
              the atoms to be enclosed by the OBB.  
              
        -----------
        Returns:
        -----------
        vertex : OBB coordinates
        eigen_vecs : transformation matrix
        vertex_no : OBB without the Sr addition
        """
        
        # Compute scatter matrix
        scat = XYZ.T @ XYZ
        # Compute singular values or eigenvalues
        eigen_vecs, _, _ = np.linalg.svd(scat)
        #_, eigen_vecs = np.linalg.eig(scat)

        # project XY onto the "principal axes"
        XYZ_proj = XYZ @ eigen_vecs

        # compute min/max values (or whatever you need)
        min_x, min_y, min_z = np.min(XYZ_proj, 0)
        max_x, max_y, max_z = np.max(XYZ_proj, 0)

        #S radius
        Sr = 1.85

        ver_proj_S = np.array([[min_x - Sr, min_y - Sr, min_z - Sr],#
                         [min_x - Sr, max_y + Sr, min_z - Sr],#
                         [max_x + Sr, min_y - Sr, min_z - Sr],
                         [max_x + Sr, max_y + Sr, min_z - Sr],
                         [min_x - Sr, min_y - Sr, max_z + Sr],#
                         [min_x - Sr, max_y + Sr, max_z + Sr],#
                         [max_x + Sr, min_y - Sr, max_z + Sr],
                         [max_x + Sr, max_y + Sr, max_z + Sr]])
        
        ver_proj = np.array([[min_x , min_y , min_z ],#
                         [min_x, max_y, min_z ],#
                         [max_x, min_y, min_z ],
                         [max_x, max_y, min_z],
                         [min_x, min_y, max_z],#
                         [min_x, max_y, max_z],#
                         [max_x, min_y, max_z],
                         [max_x, max_y, max_z]])

        # project min/max values back to the original system
        vertex = ver_proj_S @ eigen_vecs.T
        vertex_no = ver_proj @ eigen_vecs.T #vertex not oriented
        #ver = ver_proj @ eigen_vecs.T # use this with eigenvalues
        return vertex, eigen_vecs, vertex_no
    
        
class BVH:
    """Binding Volume Hierarchy.
       Object representing a Tree."""
    
    def __init__(self, left, right):
        self.left = left
        self.right = right
    
    
    def __str__(self):
        return '%s    %s' % (self.left, self.right)
    
    def create_nodes(selec): 
    
        """
        Creates every node
        -----------
        Parameters:
        -----------
        selec : List of arrays containing selection coordinates
        
        -----------
        Returns:
        -----------
        nodes: (list of lists) every level in the hierarchy and every node in each level
        """
        lower_layer = []
        for i in selec:
            node = Node(Node.Vertex_OBB(i), parents = 1, children = 0)
            lower_layer.append(node)

        combine = [lower_layer[n:n+2] for n in range(0, len(lower_layer), 2)]
        nodes = [lower_layer]

        for i in range(len(combine)):
            layer = []
            for node_list in combine:
                try:
                    layer.append(BVH.combine_nodes(node_list[0], node_list[1]))
                except:
                    layer.append(node_list[0])
            nodes.append (layer)
            combine = [layer[n:n+2] for n in range(0, len(layer), 2)]        
        return nodes
    
    def combine_nodes(a , b):
        """" Creates an OBB combining two children nodes
        -----------
        Parameters:
        -----------
        a, b : Nodes 
        
        -----------
        Returns:
        -----------
        c : new node containing a and b 
        """
        
        c_ = np.array([a.obb[2][0].tolist(),
             a.obb[2][1].tolist(),
             a.obb[2][4].tolist(),
             a.obb[2][5].tolist(),
             b.obb[2][2].tolist(),
             b.obb[2][3].tolist(),
             b.obb[2][6].tolist(),
             b.obb[2][7].tolist()])
        
        c = Node.Vertex_OBB(c_)
        
        return Node(c, parents = 1, children = 2)

In [ ]:
nodes = BVH.create_nodes(selec)
tree = BVH(BVH(BVH(nodes[0][0], nodes[0][1] ), BVH(nodes[0][2], nodes[0][3])), BVH(BVH(nodes[0][4], 0), 0))#zero when the branch is null

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(l[:,0], l[:,1], l[:,2])#, c='r')
ax.scatter(m[:,0], m[:,1], m[:,2])
ax.scatter(n[:,0], n[:,1], n[:,2])
ax.scatter(o[:,0], o[:,1], o[:,2])
ax.scatter(p[:,0], p[:,1], p[:,2])#, c = 'r')


#ctrl+/
ax.scatter(nodes[-1][0].obb[0][:,0],  nodes[-1][0].obb[0][:,1], nodes[-1][0].obb[0][:,2], c ='r')#

# ax.scatter(nodes[-2][0].obb[0][:,0],  nodes[-2][0].obb[0][:,1], nodes[-2][0].obb[0][:,2], c ='g')
# ax.scatter(nodes[-2][1].obb[0][:,0],  nodes[-2][1].obb[0][:,1], nodes[-2][1].obb[0][:,2], c ='y')

#ax.scatter(nodes[-3][0].obb[0][:,0],  nodes[-3][0].obb[0][:,1], nodes[-3][0].obb[0][:,2], c ='g')
#ax.scatter(nodes[-3][1].obb[0][:,0],  nodes[-3][1].obb[0][:,1], nodes[-3][1].obb[0][:,2], c ='y')
#ax.scatter(nodes[-3][2].obb[0][:,0],  nodes[-3][2].obb[0][:,1], nodes[-3][2].obb[0][:,2], c ='r')

# ax.scatter(nodes[0][4].obb[0][:,0],  nodes[-2][0].obb[0][:,1], nodes[-2][0].obb[0][:,2], c ='g')
# ax.scatter(nodes[0][3].obb[0][:,0],  nodes[-2][1].obb[0][:,1], nodes[-2][1].obb[0][:,2], c ='y')
# ax.scatter(nodes[0][2].obb[0][:,0],  nodes[-3][2].obb[0][:,1], nodes[-3][2].obb[0][:,2], c ='r')
# ax.scatter(nodes[0][1].obb[0][:,0],  nodes[-3][1].obb[0][:,1], nodes[-3][1].obb[0][:,2], c ='y')
# ax.scatter(nodes[0][0].obb[0][:,0],  nodes[-3][0].obb[0][:,1], nodes[-3][0].obb[0][:,2], c ='g')
